In [ ]:
import pickle
import csv
import matplotlib.pyplot as plt
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten ,Conv2D, Input, Lambda, SpatialDropout2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
import cv2
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf

K.set_image_dim_ordering("tf")

BATCH_SIZE = 32
EPOCHS = 5

FILE_PATH = "data/driving_log.csv"


def import_csv():
    data = []
    with open(FILE_PATH) as FILE:
        reader = csv.reader(FILE)
        for i in reader:
            data.append(i)
    return data


def resize_img(img):
    img = plt.imread(img)[60:135, : ]
    return img


def get_batch(data):
    indices = np.random.choice(len(data), BATCH_SIZE)
    return data.sample(n=BATCH_SIZE)


def randomize_image(data, value):
    random = np.random.randint(4)
    if (random == 0):
        path_file = data['left'][value].strip()
        shift_ang = .25
    if (random == 1 or random == 3):
        # Twice as much center images
        path_file = data['center'][value].strip()
        shift_ang = 0.
    if (random == 2):
        path_file = data['right'][value].strip()
        shift_ang = -.25

    return path_file,shift_ang


def trans_image(image,steer,trans_range = 100):
    #Translation functionto augment the steering angles and images randomly and avoid overfitting
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = steer + tr_x/trans_range*2*.2
    tr_y = 0
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(320,75))
    return image_tr,steer_ang


def generate_train(data):
    obs = 0
    while 1:
        batch = get_batch(data)
        features = np.empty([BATCH_SIZE, 75, 320, 3])
        labels = np.empty([BATCH_SIZE, 1])

        for i, value in enumerate(batch.index.values):
            x, shift = randomize_image(data, value)
            x = resize_img(x)

            x = x.reshape(x.shape[0], x.shape[1], 3)

            # Add shift to steer
            y = float(data['steer'][value]) + shift

            x, y = trans_image(x,y)

            random = np.random.randint(1)

            # Flip alomst 50% of images
            if (random == 0):
                x = np.fliplr(x)
                y = -y

            labels[i] = y
            features[i] = x

        x = np.array(features)
        y = np.array(labels)
        obs += len(x)
        yield x, y

        
def generate_valid(data):
    while 1:
        for i_line in range(len(data)):
            data = data.iloc[[i_line]].reset_index()
            x = resize_img(data['center'][0])
            x = x.reshape(1, x.shape[0], x.shape[1], 3)
            y = data['steer'][0]
            y = np.array([[y]])
            yield x, y

            
def remove_low_steering(data):
    ind = data[abs(data['steer'])<.05].index.tolist()
    rows = []
    for i in ind:
        random = np.random.randint(100)
        if random < 75:
            rows.append(i)

    data = data.drop(data.index[rows])
    print("Dropped {} rows with low steering".format(len(rows)))
    return data


def nvidia_model(img):
    shape = (img[0], img[1], 3)
    model = Sequential()

    def process(img):
        img = tf.image.resize_images(img, (66, 200))
        return img

    model.add(Lambda(process, input_shape=shape))
    model.add(Lambda(lambda x: x/255.-0.5))
    model.add(Conv2D(24, (5, 5), padding="same", activation="elu", strides=(2, 2)))
    model.add(SpatialDropout2D(0.15))
    model.add(Conv2D(36, (5, 5), padding="same", activation="elu", strides=(2, 2)))
    model.add(SpatialDropout2D(0.15))
    model.add(Conv2D(48, (5, 5), padding="valid", activation="elu", strides=(2, 2)))
    model.add(SpatialDropout2D(0.15))
    model.add(Conv2D(64, (3, 3), padding="valid", activation="elu"))
    model.add(SpatialDropout2D(0.15))
    model.add(Conv2D(64, (3, 3), padding="valid", activation="elu"))
    model.add(SpatialDropout2D(0.15))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation="elu"))
    model.add(Dense(50, activation="elu"))
    model.add(Dense(10, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    return model


# Train the network
data = pd.read_csv(FILE_PATH, index_col=False)
data.columns = ['center', 'left', 'right', 'steer', 'throttle', 'brake', 'speed']

img = resize_img(data['center'][900].strip())

model = nvidia_model(img.shape)
model.summary()
model.compile(loss='mse', optimizer='adam')

# Shuffle data
data_shuffle = data.reindex(np.random.permutation(data.index))

# Split data on a multiple of BATCH SIZE
split = (int(len(data_shuffle) * 0.85) // BATCH_SIZE) * BATCH_SIZE
train_data = data[:split]

train_data = remove_low_steering(train_data)

val_data = data[split:]
new_val = (len(val_data) // BATCH_SIZE) * BATCH_SIZE
val_data = val_data[:new_val]

samples_per_epoch = len(train_data) - BATCH_SIZE

values = model.fit_generator(generate_train(train_data), samples_per_epoch=samples_per_epoch, epochs=EPOCHS, validation_data=generate_train(val_data), validation_steps=len(val_data))

model.save('model.h5')
print("DOne ^_^ ")

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 100, 24)       1824      
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 33, 100, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 50, 36)        21636     
_________________________________________________________________
spatial_dropout2d_2 (Spatial (None, 17, 50, 36)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 23, 48)         43248     
__________

C:\Users\HEMA\Anaconda3\envs\term1\lib\site-packages\ipykernel_launcher.py:183: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=1216, validation_data=<generator..., epochs=5, steps_per_epoch=3659)`


Epoch 1/5
 672/3659 [====>.........................] - ETA: 5:46 - loss: 0.0528